# SETUP ENVIRONMENT

## MNT DRIVE AND CLONE GITHUB REPO

In [30]:
# mnt drive 
from google.colab import drive
drive.mount('/content/drive')


# Access token from drive without exposing in text (accsessible in logfiles lol. Better than nothing)
import os
import subprocess

token = open("/content/drive/MyDrive/token/colab.txt", "r").read()
tkn = token.split("=")

repo_name = "callebalik/clinical_NLP_SE.git"
cmd_string = 'git clone https://{0}@github.com/{1}'.format(tkn[1], repo_name)

!{cmd_string}
cmd_string = "" # removing the variable
print(cmd_string)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'clinical_NLP_SE' already exists and is not an empty directory.



In [31]:
# Set paths for the environment 
import os 
from pathlib import Path

drive_path = Path('/content/drive/MyDrive/bioNLP/')
code_path = Path('/content/clinical_NLP_SE/')
# sys.path.append(str(code_path)) # Let pyhton find scripts from the repo
models_path = Path('/content/drive/My Drive/bioNLP/models/')
data_path = code_path / 'data/'
corpus_path = data_path / 'raw/corpus'

In [32]:
#drive.flush_and_unmount()
#print('All changes made in this colab session should now be visible in Drive.')

In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## GET DATA AND MODELS

###Gettings the Swedish models 

https://data.kb.se/datasets/2020/10/swedish_nlp/spacy/

https://github.com/Kungbib/swedish-spacy

The models can be downloaded with the commands below. For colab this is slow and they are instead mounted from a google drive where they have been downloaded



In [34]:
# Get models from Kungbib https://github.com/Kungbib/swedish-spacy
#!wget https://data.kb.se/datasets/2020/10/swedish_nlp/spacy/sv_tagger-0.0.0.tar.gz

# Older models
#!wget https://data.kb.se/datasets/2020/10/swedish_nlp/spacy/sv_model_upos.zip
#!wget https://data.kb.se/datasets/2020/10/swedish_nlp/spacy/sv_model_xpos.zip

In [35]:
# N.B. Make sure to install the spacy-transformers extension, or the models won't work.
!pip install spacy-transformers
!pip install "/content/drive/My Drive/bioNLP/models/sv_pipeline-0.0.0.tar.gz"
import spacy


Processing /content/drive/My Drive/bioNLP/models/sv_pipeline-0.0.0.tar.gz
  Created wheel for sv-pipeline: filename=sv_pipeline-0.0.0-cp37-none-any.whl size=935375398 sha256=1b8ca2ddb64f7a2debe34e66a4f1654eb98c81006300573119f1efa581caf620
  Stored in directory: /root/.cache/pip/wheels/5f/58/a9/4f2caa352e21d9859301b0fb743111c59b70b866c9de895c40
Successfully built sv-pipeline
  Found existing installation: sv-pipeline 0.0.0
    Uninstalling sv-pipeline-0.0.0:
      Successfully uninstalled sv-pipeline-0.0.0


### Note from Kungbib

** UPDATE February 2021 **: We are adding two transformer-based models trained in spaCy 3.0. They are available to download at the same link given below.
One model is a complete pipeline with UPOS tagger, parser, sentencer, ner and lemmatizer (sv_pipeline-0.0.0.tar.gz). Unfortunately the lemmatizer is not yet trainable in spaCy, so the performance is as good as the quality of the rules/lookup tables available for Swedish (i.e. not very good). 

If you need a Swedish lemmatizer we advise you for the moment to have a look at Stanza, efselab or lemmy.ww

---


The other model is a XPOS tagger in case you need language-specific part-of-speech tags (sv_tagger-0.0.0.tar.gz).

## INITIALIZE SPACY

In [36]:
import spacy 
nlp = spacy.load("sv_pipeline")

# Analyzing

> CTRL + F8 to run all cell above and setup environment 

*Time estimate 3 min*

## Spacy PhraseMatcher 

In [29]:
from spacy.matcher import PhraseMatcher

In [118]:
# get ICD10 

""" pattern and special signs 
LDD   LLLLLL
LDD † LLLLLL
LDD * LLLLLL

Example 
P75	*	Mekoniumileus vid cystisk fibros (E84.1)
"""


'''
Positive Lookbehind (?<=\D\d\d) Assert that the Regex below matches
  \D matches any character that's not a digit (equivalent to [^0-9])
  \d matches a digit (equivalent to [0-9])
  \d matches a digit (equivalent to [0-9])
\s matches any whitespace character (equivalent to [\r\n\t\f\v ])
+? matches the previous token between one and unlimited times, as few times as possible, expanding as needed (lazy)
. matches any character (except for line terminators)
? matches the previous token between zero and one times, as many times as possible, giving back as needed (greedy)
\s matches any whitespace character (equivalent to [\r\n\t\f\v ])

Global pattern flags
g modifier: global. All matches (don't return after first match)
m modifier: multi line. Causes ^ and $ to match the begin/end of each line (not only begin/end of string)
'''

import re

regex_ddd = '/(?<=\D\d\d)\s+?.?\s/gm'
regex_dddR = '/(?<=R\d\d)\s+?.?\s/gm' 
ddd = r'(?<=\D\d\d)\s+?.?\s'
dddd = r'(?<=\D\d\d\d)\s+?.?\s'

ddddd = r'((?<=\D\d\d\d\d)|(?<=\D\d\d\d\D))\s+?.?\s'

'''
A513B	†	Syfilitisk (sekundär) alopeci (L99.8)
Y3498		Ospecificerad skadehändelse, med oklar avsikt-plats, ospecificerad-andra specificerade aktiviteter
Y3499		Ospecificerad skadehändelse, med oklar avsikt-plats, ospecificerad-aktivitet, ospecificerad
Y586A		Komplikation av vaccin mot kikhosta, enbart (P)
Y586B		Komplikation av vaccin mot difteri, kikhosta och stelkramp, kombinerat (DPT)
Y586W		Komplikation av annat kombinerat 
'''

ICD_PATH = data_path / 'raw/icd-10-se-2021-text'
file_path = ICD_PATH / 'digit3.txt'

ICD000 = []
with open(ICD_PATH / 'digit3.txt','r') as codes:
    for line in codes:
        x = re.split(ddd, line)
        ICD000.append(x[1].strip())

ICD0000 = []
with open(ICD_PATH / 'digit4.txt','r') as codes:
    for line in codes:
        x = re.split(dddd, line)
        ICD0000.append(x[1].strip())
        
ICD00000 = []
with open(ICD_PATH / 'digit5.txt','r') as codes:
    for line in codes:
        x = re.split(ddddd, line)
        ICD00000.append(x[1])



In [119]:
ICD = ICD000 + ICD0000 + ICD00000

In [121]:
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
# Only run nlp.make_doc to speed things up
icd_patterns = [nlp.make_doc(text) for text in ICD]
matcher.add("SYM", icd_patterns)

Changed in v3.0
As of spaCy v3.0, PhraseMatcher.add takes a list of patterns as the second argument (instead of a variable number of arguments). The on_match callback becomes an optional keyword argument.

```
patterns = [nlp("health care reform"), nlp("healthcare reform")]
- matcher.add("HEALTH", on_match, *patterns)
+ matcher.add("HEALTH", patterns, on_match=on_match)
``` 

In [123]:
file =  corpus_path / 'mockup-patient-records/by_johanna/originals/chart2.txt'
pre_doc = open(file, 'r').read()

doc = nlp(pre_doc)

matches = matcher(doc)
entities = []
for match_id, start, end in matches:
    span = doc[start:end]
    class_id = nlp.vocab.strings[match_id]
    print(span.text, span.start_char, span.end_char, class_id)
    ent = {"start": span.start_char, "end": span.end_char, "label": class_id}
    entities.append(ent)
# print(entities)

dyspné 27 33 SYM
acidos 1090 1096 SYM


In [87]:
width = 20 # table width
for token in doc:
    print(f"{token.text: <{width}} {token.tag_: <{width}} {token.dep_: <{width}}")

Kontaktorsak         NOUN                 ROOT                
:                    PUNCT                punct               
Dyspné               NOUN                 ROOT                

                    PUNCT                ROOT                
Tidigare             ADJ                  amod                
sjukdomar            NOUN                 ROOT                
:                    PUNCT                punct               
Insulin              ADJ                  amod                
och                  CCONJ                cc                  
tablettbehandlad     ADJ                  conj                
DM2                  NUM                  ROOT                
.                    PUNCT                punct               
Tablettbehandlad     ADJ                  amod                
hypertoni            NOUN                 ROOT                
.                    PUNCT                punct               
Höga                 ADJ                  amod         

In [89]:
import pandas as pd

cols = ("text", "lemma", "POS", "explain", "stopword")
rows = []

for t in doc:
    row = [t.text, t.lemma_, t.pos_, spacy.explain(t.pos_), t.is_stop]
    rows.append(row)

df = pd.DataFrame(rows, columns=cols)
    
df



ValueError: ignored

### Entiites

In [85]:
from spacy import displacy

test = [{"text": doc,
       "ents": entities,
       "title": None}]
html = displacy.render(test, style="ent", manual=True, jupyter=True)

AttributeError: ignored

In [ ]:
for ent in doc.ents:
    print(ent.text, ent.label_)

In [ ]:
from spacy import displacy

displacy.render(doc, style="dep", jupyter=True)

* raw text
* lemma – a root form of the word
* part of speech
* a flag for whether the word is a stopword – i.e., a common word that may be filtered out

In [ ]:
for sent in doc.sents:
    print(">", sent)

> Kontaktorsak:
> Dyspné
> 

> Tidigare sjukdomar:
> Insulin och tablettbehandlad DM2.
> Tablettbehandlad hypertoni.
> Höga blodfetter.
> 

> Socialt:
> Ensamboende, hemtjänst * 1.

> Aktuellt:
> Insjuknat för tre dagar sedan med andnöd vid gång.
> Tillkomst av hosta under gårdagen.
> Ej noterat feber, men ej eller mätt tempen.
> Idag nedsatt AT, svårt att gå.
> Tagit sina mediciner som vanligt.
> Sökt VC som hänvisat hit pga destruering.
> Inte haft några utslag.
> Ej kontakt med någon annan som varit sjuk.
> Ej varit ute och rest.
> Ingen smärta vid miktion, buksmärta eller ÖNH symptom.
> Idag sämre, kan ej mobilisera sig i hemmet.
> Inkommer på remiss.
> Inga kräkningar eller GI symptom.
> 

> Status:
> 

> AT:
> Gott, opåverkad i vila, ingen samtalsdyspné.
> 

> Cor:
> Normofrekvent regelbunden rytm, inga hörbara bi eller blåsljud.

> Pulm:
> I stort vesikulära antingsljud bilat, dämpat basalt höger.
> Inget förlängt expirium.
> 

> Buk:
> Överallt muk och oöm.
> Ingen lokal perito

In [ ]:
for chunk in doc.noun_chunks:
    print(chunk.text)

Kontaktorsak:
Dyspné
Tidigare sjukdomar:
Tablettbehandlad hypertoni.
Höga blodfetter.
, hemtjänst
Aktuellt:
Tillkomst av hosta under gårdagen.
feber, men ej eller mätt tempen
Idag nedsatt AT, svårt att gå.
sina
VC som hänvisat hit pga destruering
några utslag
Ej kontakt med någon annan som varit sjuk.
och rest
Ingen smärta vid miktion, buksmärta eller ÖNH symptom.
sig
Inga kräkningar eller GI symptom.
Status:
AT:
Cor:
Normofrekvent regelbunden rytm, inga hörbara bi eller blåsljud.

Pulm:
I stort vesikulära antingsljud bilat, dämpat basalt höger.
Inget förlängt expirium.
Buk:
Överallt muk och oöm.
Ingen lokal peritonit.
Lokalstatus:
Underben
, inga pittingödem
BT: 117/80

Sat: 92% på 2l O2
AF: 28

Temp: 38.8

Lab:
Blodgas utan respiratorisk acidos, Ph 7,4, CRP 118, diskret förhöjt BE.
Normalt laktat
Tobak:
EKG:
Sinusrym med kammarfrekvens 98, inga urakuta ischemiska tecken.


Bedömning:
I första hand misstanke om penumoni eller COVID-19, inlägges infektionsavdelning.
cefotaxim
Nya prove

In [ ]:
displacy.render(doc, style="ent", jupyter=True)